<a href="https://colab.research.google.com/github/RegNLP/RePASs/blob/main/RIRAG_SimplificityScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordfreq textstat nltk torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [22]:
from wordfreq import zipf_frequency
import textstat, numpy as np, nltk, torch, json, csv, os

# Download required NLTK data if not already present.
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# A helper function to convert a shift value to a score.
def shift_to_score(shift, target_shift, right_slope=0.25):
    if shift <= target_shift:
        score = shift / (target_shift + 0.001)
    else:
        score = 1.0 - right_slope * (shift - target_shift) / (target_shift + 0.001)
    return np.clip(score, 0, 1.0)

#############################################
# Lexical Simplicity Score
#############################################
class SimplicityLexicalScore:
    def __init__(self, target_shift=0.4, word_change_ratio=0.1):
        self.target_shift = target_shift
        self.word_change_ratio = word_change_ratio  # Expected fraction of word changes
        self.stopws = set(nltk.corpus.stopwords.words("english") + ["might", "would", "``"])

    def word_score_func(self, w):
        return zipf_frequency(w, 'en', wordlist="large")

    def is_good_word(self, w):
        if "'" in w:
            return False
        if len(w) > 30 or len(w) == 1:
            return False
        if w.lower() in self.stopws:
            return False
        if all(c.isdigit() for c in w):
            return False
        return True

    def vocab_shift_score(self, txt1, txt2, printing=False):
        words1 = nltk.tokenize.word_tokenize(txt1)
        words2 = nltk.tokenize.word_tokenize(txt2)
        words1 = set([w.lower() for w in words1 if self.is_good_word(w)])
        words2 = set([w.lower() for w in words2 if self.is_good_word(w)])
        removed_words = words1 - words2
        added_words = words2 - words1
        target_n_words = int(self.word_change_ratio * txt1.count(" "))

        vocab_shift = 0.0
        if target_n_words == 0:
            vocab_shift = 1.0  # No expected shifts yet
        elif len(removed_words) > 0 and len(added_words) > 0:
            added_words_zipfs = [{"w": w, "zipf": self.word_score_func(w)} for w in added_words]
            removed_words_zipfs = [{"w": w, "zipf": self.word_score_func(w)} for w in removed_words]
            added_words_zipfs = sorted(added_words_zipfs, key=lambda x: x['zipf'])
            removed_words_zipfs = sorted(removed_words_zipfs, key=lambda x: x['zipf'])[:target_n_words]

            removed_avg_zipfs = np.mean([x['zipf'] for x in removed_words_zipfs[:target_n_words]])
            added_avg_zipfs = np.mean([x['zipf'] for x in added_words_zipfs[:min(target_n_words, len(removed_words_zipfs))]])
            if printing:
                print("Desired # word swaps: %d" % (target_n_words))
                print("[Avg Zipf: %.3f] Added words:" % (added_avg_zipfs), added_words_zipfs)
                print("[Avg Zipf: %.3f] Removed words:" % (removed_avg_zipfs), removed_words_zipfs)

            vocab_shift = (added_avg_zipfs - removed_avg_zipfs) * len(removed_words_zipfs) / target_n_words

        return vocab_shift, len(added_words), len(removed_words)

    def score(self, sources, generateds, partial=False, printing=False, **kwargs):
        scores = []
        vshifts = []
        n_adds, n_dels = [], []
        for source, generated in zip(sources, generateds):
            if partial:
                source = " ".join(source.split(" ")[:generated.count(" ")])
            vshift, n_add, n_del = self.vocab_shift_score(source, generated, printing=printing)
            score = shift_to_score(vshift, self.target_shift)
            vshifts.append(vshift)
            scores.append(score)
            n_adds.append(n_add)
            n_dels.append(n_del)
        scores = torch.FloatTensor(scores)
        scores = (0.3 + torch.clamp(scores, 0.05, 1.0) * 0.7).tolist()
        if printing:
            print("[vshift]", scores)
        return {"scores": scores, "n_w_adds": n_adds, "n_w_dels": n_dels, "vshifts": vshifts}

#############################################
# Syntactic Simplicity Score
#############################################
class SimplicitySyntacticScore:
    def __init__(self):
        pass

    def rsource2target_shift(self, rsource):
        # A piecewise linear function: the higher the source grade, the more simplification is expected.
        if rsource <= 4.0:
            return 0
        elif rsource <= 12.0:
            return (rsource - 3) * 0.5
        return 4.5 + (rsource - 12) * 0.83

    def readability_shift_score(self, txt1, txt2):
        score1 = textstat.flesch_kincaid_grade(txt1)
        score2 = textstat.flesch_kincaid_grade(txt2)
        return score1, score2

    def score(self, sources, generateds, partial=False, printing=False, **kwargs):
        scores = []
        rshifts, rsources, rtargets = [], [], []
        for source, generated in zip(sources, generateds):
            if partial:
                source = " ".join(source.split(" ")[:generated.count(" ")])
            rsource, rtarget = self.readability_shift_score(source, generated)
            rshift = rsource - rtarget
            target_shift = self.rsource2target_shift(rsource)
            score = shift_to_score(rshift, target_shift)
            rshifts.append(rshift)
            rsources.append(rsource)
            rtargets.append(rtarget)
            scores.append(score)
        scores = torch.FloatTensor(scores)
        scores = (0.05 + torch.clamp(scores, 0.02, 1.0) * 0.95).tolist()
        if printing:
            print("[rshift]", scores)
        return {"scores": scores, "rshifts": rshifts, "rsources": rsources, "rtargets": rtargets}

#############################################
# Main: Compute Simplicity Scores
#############################################
def main():
    # Update these variables with your desired inputs.
    input_json_file = "/content/drive/Othercomputers/MBZUAI/MBZUAI/RIRAG System Submission/12_AUEB NLP Group/submission3/no_gen.json"      # e.g., "data/input.json"
    output_folder_path = "/content/drive/Othercomputers/MBZUAI/MBZUAI/RIRAG Task 2 Simplification Evaluations"      # e.g., "results"
    method_name = "lexico_syntactic_simplicity_scores"
    team_name = "12_AUEB NLP _3"

    # Create final output folder by joining the output folder path, method name, and team name.
    final_output_folder = os.path.join(output_folder_path, method_name, team_name)
    os.makedirs(final_output_folder, exist_ok=True)

    # Load JSON data from the file.
    with open(input_json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # If the loaded data is a single JSON object, wrap it in a list.
    if not isinstance(data, list):
        data = [data]

    # Initialize scorers.
    lexical_scorer = SimplicityLexicalScore(target_shift=0.4, word_change_ratio=0.1)
    syntactic_scorer = SimplicitySyntacticScore()

    rows = []
    # Initialize accumulators.
    total_lexical_score = 0
    total_n_adds = 0
    total_n_dels = 0
    total_vshift = 0

    total_syntactic_score = 0
    total_rshift = 0
    total_rsource = 0
    total_rtarget = 0

    count = 0

    for item in data:
        question_id = item.get("QuestionID", "")
        retrieved_passages = item.get("RetrievedPassages", [])
        if not retrieved_passages:
            print(f"Skipping {question_id} due to empty RetrievedPassages")
            raw_text = "No Retrieved Passages Available"
        else:
            raw_text = " ".join(retrieved_passages).strip()

        simplified_text = item.get("Answer", "").strip()
        if not simplified_text:
            print(f"Skipping {question_id} due to empty Answer")
            simplified_text = "No Answer Provided"

        lexical_results = lexical_scorer.score([raw_text], [simplified_text], printing=False)
        syntactic_results = syntactic_scorer.score([raw_text], [simplified_text], printing=False)

        lexical_score = lexical_results["scores"][0]
        n_w_adds = lexical_results["n_w_adds"][0]
        n_w_dels = lexical_results["n_w_dels"][0]
        vshift = lexical_results["vshifts"][0]

        syntactic_score = syntactic_results["scores"][0]
        rshift = syntactic_results["rshifts"][0]
        rsource = syntactic_results["rsources"][0]
        rtarget = syntactic_results["rtargets"][0]

        # Print the results for this item.
        #print(f"Processed QuestionID: {question_id}")
        #print(f"  Lexical Simplicity Score: {lexical_score:.4f} (vshift: {vshift:.4f}, adds: {n_w_adds}, dels: {n_w_dels})")
        #print(f"  Syntactic Simplicity Score: {syntactic_score:.4f} (rshift: {rshift:.4f}, source: {rsource:.4f}, target: {rtarget:.4f})\n")

        row = {
            "QuestionID": question_id,
            "LexicalSimplicityScore": lexical_score,
            "n_w_adds": n_w_adds,
            "n_w_dels": n_w_dels,
            "vshift": vshift,
            "SyntacticSimplicityScore": syntactic_score,
            "rshift": rshift,
            "rsource": rsource,
            "rtarget": rtarget
        }
        rows.append(row)

        total_lexical_score += lexical_score
        total_n_adds += n_w_adds
        total_n_dels += n_w_dels
        total_vshift += vshift

        total_syntactic_score += syntactic_score
        total_rshift += rshift
        total_rsource += rsource
        total_rtarget += rtarget

        count += 1

    #print(f"Processed {count} items.\n")

    # Write individual item results to a CSV file.
    csv_file_path = os.path.join(final_output_folder, "simplicity_scores.csv")
    fieldnames = ["QuestionID", "LexicalSimplicityScore", "n_w_adds", "n_w_dels", "vshift",
                  "SyntacticSimplicityScore", "rshift", "rsource", "rtarget"]
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    # Compute average scores across all items.
    if count > 0:
        avg_lexical_score = total_lexical_score / count
        avg_n_adds = total_n_adds / count
        avg_n_dels = total_n_dels / count
        avg_vshift = total_vshift / count

        avg_syntactic_score = total_syntactic_score / count
        avg_rshift = total_rshift / count
        avg_rsource = total_rsource / count
        avg_rtarget = total_rtarget / count
    else:
        avg_lexical_score = avg_n_adds = avg_n_dels = avg_vshift = 0
        avg_syntactic_score = avg_rshift = avg_rsource = avg_rtarget = 0

    # Write average scores to a TXT file.
    txt_file_path = os.path.join(final_output_folder, "average_scores.txt")
    with open(txt_file_path, 'w', encoding='utf-8') as txtfile:
        txtfile.write("Average Simplicity Scores:\n\n")
        txtfile.write("Lexical Simplicity:\n")
        txtfile.write(f"Average Lexical Simplicity Score: {avg_lexical_score:.4f}\n")
        txtfile.write(f"Average n_w_adds: {avg_n_adds:.2f}\n")
        txtfile.write(f"Average n_w_dels: {avg_n_dels:.2f}\n")
        txtfile.write(f"Average vshift: {avg_vshift:.4f}\n\n")
        txtfile.write("Syntactic Simplicity:\n")
        txtfile.write(f"Average Syntactic Simplicity Score: {avg_syntactic_score:.4f}\n")
        txtfile.write(f"Average rshift: {avg_rshift:.4f}\n")
        txtfile.write(f"Average rsource: {avg_rsource:.4f}\n")
        txtfile.write(f"Average rtarget: {avg_rtarget:.4f}\n")

        # Print the file content
    with open(txt_file_path, 'r', encoding='utf-8') as txtfile:
        print("\n--- Results ---")
        print(txtfile.read())  # Print the results to the console

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



--- Results ---
Average Simplicity Scores:

Lexical Simplicity:
Average Lexical Simplicity Score: 0.3404
Average n_w_adds: 0.00
Average n_w_dels: 3.30
Average vshift: 0.0135

Syntactic Simplicity:
Average Syntactic Simplicity Score: 0.0783
Average rshift: -0.8229
Average rsource: 24.8395
Average rtarget: 25.6623



In [ ]:
from wordfreq import zipf_frequency
import textstat, numpy as np, nltk, torch, json, csv, os

# Download required NLTK data if not already present.
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

def shift_to_score(shift, target_shift, right_slope=0.25):
    if shift <= target_shift:
        score = shift / (target_shift + 0.001)
    else:
        score = 1.0 - right_slope * (shift - target_shift) / (target_shift + 0.001)
    return np.clip(score, 0, 1.0)

#############################################
# Lexical Simplicity Score
#############################################
class SimplicityLexicalScore:
    def __init__(self, target_shift=0.4, word_change_ratio=0.1):
        self.target_shift = target_shift
        self.word_change_ratio = word_change_ratio  # Expected fraction of word changes
        self.stopws = set(nltk.corpus.stopwords.words("english") + ["might", "would", "``"])

    def word_score_func(self, w):
        return zipf_frequency(w, 'en', wordlist="large")

    def is_good_word(self, w):
        if "'" in w:
            return False
        if len(w) > 30 or len(w) == 1:
            return False
        if w.lower() in self.stopws:
            return False
        if all(c.isdigit() for c in w):
            return False
        return True

    def compute_text_lexical_simplicity(self, text):
        """Compute the average Zipf frequency of the good words in text."""
        words = nltk.tokenize.word_tokenize(text)
        good_words = [w.lower() for w in words if self.is_good_word(w)]
        if not good_words:
            return 0.0
        avg_zipf = np.mean([self.word_score_func(w) for w in good_words])
        return avg_zipf

    def score(self, sources, generateds, partial=False, printing=False, **kwargs):
        results = []
        for source, generated in zip(sources, generateds):
            if partial:
                source = " ".join(source.split(" ")[:generated.count(" ")])
            source_lexical = self.compute_text_lexical_simplicity(source)
            generated_lexical = self.compute_text_lexical_simplicity(generated)
            results.append({
                "source_lexical": source_lexical,
                "generated_lexical": generated_lexical,
                "difference": generated_lexical - source_lexical
            })
        if printing:
            print("[lexical]", results)
        return results

#############################################
# Syntactic Simplicity Score
#############################################
class SimplicitySyntacticScore:
    def __init__(self, max_grade=30):
        self.max_grade = max_grade

    def compute_syntactic_simplicity(self, text):
        """Compute the Flesch-Kincaid grade and convert it to a simplicity score.
           Here, lower grade means simpler text. We convert using:
              simplicity = (max_grade - grade) / max_grade.
           The value is clipped to [0, 1]."""
        grade = textstat.flesch_kincaid_grade(text)
        simplicity = (self.max_grade - grade) / self.max_grade
        simplicity = max(0, min(1, simplicity))
        return grade, simplicity

    def score(self, sources, generateds, partial=False, printing=False, **kwargs):
        results = []
        for source, generated in zip(sources, generateds):
            if partial:
                source = " ".join(source.split(" ")[:generated.count(" ")])
            source_grade, source_simplicity = self.compute_syntactic_simplicity(source)
            generated_grade, generated_simplicity = self.compute_syntactic_simplicity(generated)
            diff_grade = source_grade - generated_grade  # positive if generated is simpler (lower grade)
            results.append({
                "source_grade": source_grade,
                "source_simplicity": source_simplicity,
                "generated_grade": generated_grade,
                "generated_simplicity": generated_simplicity,
                "grade_difference": diff_grade
            })
        if printing:
            print("[syntactic]", results)
        return results

#############################################
# Main: Compute Simplicity Scores
#############################################
def main():
    # Update these variables with your desired inputs.
    input_json_file = "/content/drive/Othercomputers/MBZUAI/MBZUAI/RIRAG System Submission/0_Baseline/retrieval_results.passage_only_bm25_answers.json"      # e.g., "data/input.json"
    output_folder_path = "/content/drive/Othercomputers/MBZUAI/MBZUAI/RIRAG Task 2 Simplification Evaluations"      # e.g., "results"
    method_name = "lexico_syntactic_simplicity_scores_2"
    team_name = "0_Baseline/only_bm25"

    # Create final output folder by joining the output folder path, method name, and team name.
    final_output_folder = os.path.join(output_folder_path, method_name, team_name)
    os.makedirs(final_output_folder, exist_ok=True)

    # Load JSON data from the file.
    with open(input_json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    if not isinstance(data, list):
        data = [data]

    # Initialize scorers.
    lexical_scorer = SimplicityLexicalScore(target_shift=0.4, word_change_ratio=0.1)
    syntactic_scorer = SimplicitySyntacticScore(max_grade=30)

    rows = []
    # Accumulators for averages.
    total_source_lex = 0
    total_generated_lex = 0

    total_source_grade = 0
    total_source_simplicity = 0
    total_generated_grade = 0
    total_generated_simplicity = 0
    total_grade_diff = 0

    count = 0

    for item in data:
        question_id = item.get("QuestionID", "")
        retrieved_passages = item.get("RetrievedPassages", [])
        raw_text = " ".join(retrieved_passages)
        simplified_text = item.get("Answer", "")

        lexical_results = lexical_scorer.score([raw_text], [simplified_text], printing=False)[0]
        syntactic_results = syntactic_scorer.score([raw_text], [simplified_text], printing=False)[0]

        source_lex = lexical_results["source_lexical"]
        generated_lex = lexical_results["generated_lexical"]

        source_grade = syntactic_results["source_grade"]
        source_simplicity = syntactic_results["source_simplicity"]
        generated_grade = syntactic_results["generated_grade"]
        generated_simplicity = syntactic_results["generated_simplicity"]
        grade_diff = syntactic_results["grade_difference"]

        # Print the results for this item.
        print(f"Processed QuestionID: {question_id}")
        print(f"  Source Lexical Simplicity: {source_lex:.4f}")
        print(f"  Generated Lexical Simplicity: {generated_lex:.4f}")
        print(f"  Source Syntactic Grade: {source_grade:.4f} -> Syntactic Simplicity: {source_simplicity:.4f}")
        print(f"  Generated Syntactic Grade: {generated_grade:.4f} -> Syntactic Simplicity: {generated_simplicity:.4f}")
        print(f"  Grade Difference (source - generated): {grade_diff:.4f}\n")

        row = {
            "QuestionID": question_id,
            "SourceLexicalSimplicity": source_lex,
            "GeneratedLexicalSimplicity": generated_lex,
            "SourceSyntacticGrade": source_grade,
            "SourceSyntacticSimplicity": source_simplicity,
            "GeneratedSyntacticGrade": generated_grade,
            "GeneratedSyntacticSimplicity": generated_simplicity,
            "GradeDifference": grade_diff
        }
        rows.append(row)

        total_source_lex += source_lex
        total_generated_lex += generated_lex

        total_source_grade += source_grade
        total_source_simplicity += source_simplicity
        total_generated_grade += generated_grade
        total_generated_simplicity += generated_simplicity
        total_grade_diff += grade_diff

        count += 1

    print(f"Processed {count} items.\n")

    # Write individual item results to a CSV file.
    csv_file_path = os.path.join(final_output_folder, "simplicity_scores.csv")
    fieldnames = ["QuestionID", "SourceLexicalSimplicity", "GeneratedLexicalSimplicity",
                  "SourceSyntacticGrade", "SourceSyntacticSimplicity",
                  "GeneratedSyntacticGrade", "GeneratedSyntacticSimplicity",
                  "GradeDifference"]
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    # Compute average scores across all items.
    if count > 0:
        avg_source_lex = total_source_lex / count
        avg_generated_lex = total_generated_lex / count

        avg_source_grade = total_source_grade / count
        avg_source_simplicity = total_source_simplicity / count
        avg_generated_grade = total_generated_grade / count
        avg_generated_simplicity = total_generated_simplicity / count
        avg_grade_diff = total_grade_diff / count
    else:
        avg_source_lex = avg_generated_lex = 0
        avg_source_grade = avg_source_simplicity = avg_generated_grade = avg_generated_simplicity = avg_grade_diff = 0

    print("Average Scores:")
    print(f"  Lexical Simplicity:")
    print(f"    Source: {avg_source_lex:.4f}")
    print(f"    Generated: {avg_generated_lex:.4f}")
    print(f"  Syntactic Simplicity (converted from Flesch-Kincaid):")
    print(f"    Average Source Grade: {avg_source_grade:.4f} -> Simplicity: {avg_source_simplicity:.4f}")
    print(f"    Average Generated Grade: {avg_generated_grade:.4f} -> Simplicity: {avg_generated_simplicity:.4f}")
    print(f"    Average Grade Difference (source - generated): {avg_grade_diff:.4f}\n")

    # Write average scores to a TXT file.
    txt_file_path = os.path.join(final_output_folder, "average_scores.txt")
    with open(txt_file_path, 'w', encoding='utf-8') as txtfile:
        txtfile.write("Average Simplicity Scores:\n\n")
        txtfile.write("Lexical Simplicity:\n")
        txtfile.write(f"  Source Lexical Simplicity: {avg_source_lex:.4f}\n")
        txtfile.write(f"  Generated Lexical Simplicity: {avg_generated_lex:.4f}\n\n")
        txtfile.write("Syntactic Simplicity (converted from Flesch-Kincaid):\n")
        txtfile.write(f"  Average Source Grade: {avg_source_grade:.4f} -> Simplicity: {avg_source_simplicity:.4f}\n")
        txtfile.write(f"  Average Generated Grade: {avg_generated_grade:.4f} -> Simplicity: {avg_generated_simplicity:.4f}\n")
        txtfile.write(f"  Average Grade Difference (source - generated): {avg_grade_diff:.4f}\n")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Processed QuestionID: f10933cb-8e29-4f37-82ab-0ea23af317ac
  Source Lexical Simplicity: 4.4506
  Generated Lexical Simplicity: 4.4410
  Source Syntactic Grade: 21.2000 -> Syntactic Simplicity: 0.2933
  Generated Syntactic Grade: 18.8000 -> Syntactic Simplicity: 0.3733
  Grade Difference (source - generated): 2.4000

Processed QuestionID: c62ccb81-1712-4b11-8a73-26dcbe710272
  Source Lexical Simplicity: 4.4576
  Generated Lexical Simplicity: 4.5870
  Source Syntactic Grade: 9.7000 -> Syntactic Simplicity: 0.6767
  Generated Syntactic Grade: 18.3000 -> Syntactic Simplicity: 0.3900
  Grade Difference (source - generated): -8.6000

Processed QuestionID: 5abfd03e-38d5-4428-a594-e323098cd7d9
  Source Lexical Simplicity: 4.3178
  Generated Lexical Simplicity: 4.4408
  Source Syntactic Grade: 36.1000 -> Syntactic Simplicity: 0.0000
  Generated Syntactic Grade: 20.3000 -> Syntactic Simplicity: 0.3233
  Grade Difference (source - generated): 15.8000

Processed QuestionID: 6957553b-0fc3-4128-8b41